#### [Notebook 2 Quick Start](https://confluence.shopee.io/x/7O0lhw)
#### Common Use Cases and Solution Using Notebook 2
- [Multi Kernels in Notebook 2](https://confluence.shopee.io/x/4Od2hw)
- [Execute HDFS command](https://confluence.shopee.io/x/P5w8hw)
- [Browse, upload and download from Data Infra S3](https://confluence.shopee.io/x/xo9Ohw)
- [Setting up 3rd party packages for your Notebook 2](https://confluence.shopee.io/x/JZFOhw)
- [Develop and setup user defined function or library](https://confluence.shopee.io/x/hpjehg)
- [Execute Presto/Trino query directly in Python Kernel Notebook](https://confluence.shopee.io/x/WpMlhw)

> Note: "Python on K8S" Kernel cannot execute Spark functions. ***If you want to use Spark, change kernel to Spark***. [How to change Kernel](https://confluence.shopee.io/x/4Od2hw#MultiKernelsinNotebook2-SetKernelwhencreatingnewNotebook2file)

In [7]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType, StructType, StructField
import re

In [17]:
script = spark.sql('''
SELECT task_owner, workflow_code, task_name, exe_content as query
FROM data_metamart.dwd_data_suite_task_instance_di_ph_view
WHERE task_owner LIKE '%jayme%'
  AND task_name IS NOT NULL
  AND task_name = 'output_a_without_parcel_hub'
  limit 1
''')


In [18]:
script.show()

Interrupted by user


In [19]:

# Define a function to extract standardized comments
def extract_standard_comments(query):
    pattern = re.compile(r'-- (\d{1,2}/\d{1,2}/\d{4}) \| ([\w\s]+) \| (.+)')
    matches = pattern.findall(query)
    comments = []
    for match in matches:
        comments.append((match[0], match[1].strip(), match[2].strip()))
    return comments


In [20]:

# Define UDF to apply the extraction function
extract_comments_udf = udf(lambda x: extract_standard_comments(x), StringType())

# Apply the UDF and explode the results to create a new DataFrame
comments_schema = StructType([
    StructField("date", StringType(), True),
    StructField("author", StringType(), True),
    StructField("description", StringType(), True)
])

# Extract comments and create new DataFrame
comments_rdd = script.rdd.flatMap(lambda row: [(row['task_owner'], row['workflow_code'], row['task_name'], *comment) for comment in extract_standard_comments(row['query'])])
comments_df = spark.createDataFrame(comments_rdd, schema=StructType([
    StructField("task_owner", StringType(), True),
    StructField("workflow_code", StringType(), True),
    StructField("task_name", StringType(), True),
    StructField("date", StringType(), True),
    StructField("author", StringType(), True),
    StructField("description", StringType(), True)
]))


comments_df.show(truncate=False)

+----------+----------------+---------------------------+---------+--------------+-----------------------------------------------------------------------------------------------------------------------------------------------+
|task_owner|workflow_code   |task_name                  |date     |author        |description                                                                                                                                    |
+----------+----------------+---------------------------+---------+--------------+-----------------------------------------------------------------------------------------------------------------------------------------------+
|john.jayme|phbi_ops_6095616|output_a_without_parcel_hub|4/12/2024|Tristan Medina|Added and updated necessary columns for points logic                                                                                           |
|john.jayme|phbi_ops_6095616|output_a_without_parcel_hub|4/23/2024|John Jayme    |Added cole